![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/08.2.Clinical_Text_Classification_with_SparkNLP.ipynb)

# Colab Setup

In [ ]:
! pip install -q johnsnowlabs 

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pyspark.sql.functions as F

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start(hardware_target="gpu")

# Classifiers

The below classifiers will be used in this notebook.ClassifierDL, MultiClassifierDL, and GenericClassifier will be trained using healthcare_100d, embeddingd_clinical, and bert sentence embeddings(sbiobert_base_cased_mli). DocumentLogRegClassifier accepts tokens, so sentence embeddings are not utilized during DocumentLogRegClassifier training.

## ClassifierDL

ClassifierDL is a generic Multi-class Text Classification annotator. ClassifierDL uses the state-of-the-art Universal Sentence Encoder as an input for text classifications. The ClassifierDL annotator uses a deep learning model (DNNs) built inside TensorFlow and supports up to 100 classes. For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#classifierdl).


##  MultiClassifierDL

MultiClassifierDL is a Multi-label Text Classification annotator.MultiClassifierDL uses a Bidirectional GRU with a convolutional model built inside TensorFlow and supports up to 100 classes.  For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#multiclassifierdl).


## GenericClassifier

GenericClassifier is a TensorFlow model for the generic classification of feature vectors in Healthcare  Lİbrary. It takes FEATURE_VECTOR annotations from FeaturesAssembler as input, classifies them, and outputs CATEGORY annotations. Please see [the link](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericclassifier) for more information.

## GenericLogRegClassifierApproach

`GenericLogRegClassifierApproach` is a derivative of GenericClassifier which implements a multinomial *Logistic Regression*. This is a single layer neural network with the logistic function at the output. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores varying between 0 and 1. Training data requires "text" and their "label" columns only and the trained model will be a `GenericLogRegClassifierModel()`.


## GenericSVMClassifierApproach

`GenericSVMClassifierApproach` is a derivative of GenericClassifier which implements *SVM (Support Vector Machine)* classification. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores. The scores are standardized using the logistic function so that they vary between 0 and 1. Taining data requires "text" and their "label" columns only and the trained model will be a `GenericSVMClassifierModel()`

## DocumentLogRegClassifier

DocumentLogRegClassifier is a model to classify documents with a Logarithmic Regression algorithm in Healthcare  Library. Training data requires columns for text and labels. The result is a trained DocumentLogRegClassifierModel. you can get more info [here](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#documentlogregclassifier). 


# ADE Dataset

### Data Preprocessing

In [ ]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

**ADE-Negative Dataset**

In [ ]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg.head()

,col1
0,6460590 NEG Clioquinol intoxication occurring ...
1,"8600337 NEG ""Retinoic acid syndrome"" was preve..."
2,8402502 NEG BACKGROUND: External beam radiatio...
3,"8700794 NEG Although the enuresis ceased, she ..."
4,17662448 NEG A 42-year-old woman had uneventfu...


In [ ]:
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


**ADE Positive Dataset**

In [ ]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos.head()

,0,1,2,3,4,5,6,7
0,10030778,Intravenous azithromycin-induced ototoxicity.,ototoxicity,43,54,azithromycin,22,34
1,10048291,"Immobilization, while Paget's bone disease was...",increased calcium-release,960,985,dihydrotachysterol,908,926
2,10048291,Unaccountable severe hypercalcemia in a patien...,hypercalcemia,31,44,dihydrotachysterol,94,112
3,10082597,METHODS: We report two cases of pseudoporphyri...,pseudoporphyria,620,635,naproxen,646,654
4,10082597,METHODS: We report two cases of pseudoporphyri...,pseudoporphyria,620,635,oxaprozin,659,668


In [ ]:
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was...",pos
2,Unaccountable severe hypercalcemia in a patien...,pos
3,METHODS: We report two cases of pseudoporphyri...,pos
4,METHODS: We report two cases of pseudoporphyri...,pos


**Merging Positive and Negative dataset**

In [ ]:
ade_df= pd.concat([df_neg, df_pos])
ade_df.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


In [ ]:
ade_df["category"].value_counts()

neg    16695
pos     6821
Name: category, dtype: int64

In [ ]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23516 entries, 0 to 6820
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB


We take 30% of the data to make a faster run. You can use all data for better scores.

In [ ]:
spark_df = spark.createDataFrame(ade_df).sample(0.3, 3) # limit the data

trainingData, testData = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 5531
Test Dataset Count: 1427


In [ ]:
spark_df.groupBy("category").count().show()

+--------+-----+
|category|count|
+--------+-----+
|     pos| 2026|
|     neg| 4932|
+--------+-----+



In [ ]:
spark_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)



In [ ]:
spark_df.head(3)

[Row(text=' Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.', category='neg'),
 Row(text=' A 42-year-old woman had uneventful bilateral laser-assisted subepithelial keratectomy (LASEK) to correct myopia.', category='neg'),
 Row(text=' A 16-year-old girl with erosive, polyarticular JRA showed no detectable change in her articular disease following nine exchanges.', category='neg')]

## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



Now we will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| "Syndrome malin"-like symptoms probably due to interaction between neurolept...|     neg|[{sentence_embeddings, 0, 109,  "Syndrome malin"-like symptoms probably due t...|
| 'Bail-out' bivalirudin use in patients with thrombotic complications unrespo...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin use in patients with t...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| 1 previously untreated patient with plasma-cell leukaemia and 8 patients wit...|     neg|[{sentence_embeddings, 0, 166,  1 previously untreated patient with plasma-ce...|
| A 15-year-old boy had temporary hypertropia, supraduction deficit, ipsilater...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had temporary hypertropia, ...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [ ]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [ ]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
log_folder="ADE_logs_healthcare_100d"
!mkdir -p $log_folder

### ClassifierDLApproach

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = nlp.Pipeline(
    stages = [
        classifier_dl
    ])

In [ ]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
!cat $log_folder/ClassifierDLApproach_*

Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 16 - training_examples: 5531 - classes: 2
Epoch 0/30 - 3.73s - loss: 189.75111 - acc: 0.7523056 - batches: 346
Epoch 1/30 - 0.69s - loss: 175.62492 - acc: 0.8018445 - batches: 346
Epoch 2/30 - 0.67s - loss: 169.34398 - acc: 0.82203555 - batches: 346
Epoch 3/30 - 0.68s - loss: 164.51093 - acc: 0.83109355 - batches: 346
Epoch 4/30 - 0.71s - loss: 161.37875 - acc: 0.84214425 - batches: 346
Epoch 5/30 - 0.66s - loss: 159.21469 - acc: 0.848666 - batches: 346
Epoch 6/30 - 0.65s - loss: 157.47668 - acc: 0.85772395 - batches: 346
Epoch 7/30 - 0.66s - loss: 156.04858 - acc: 0.8615283 - batches: 346
Epoch 8/30 - 0.65s - loss: 154.84445 - acc: 0.864608 - batches: 346
Epoch 9/30 - 0.65s - loss: 153.76982 - acc: 0.8669631 - batches: 346
Epoch 10/30 - 0.64s - loss: 152.77791 - acc: 0.87022394 - batches: 346
Epoch 11/30 - 0.64s - loss: 151.87325 - acc: 0.87475294 - batches: 346
Epoch 12/30 - 0.63s - loss: 151.10619 - acc: 0.8762022 - 

In [ ]:
preds = clfDL_model_hc100.transform(testData_with_embeddings).cache()

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    

In [ ]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.86      0.90      0.88       982
         pos       0.76      0.69      0.72       445

    accuracy                           0.83      1427
   macro avg       0.81      0.79      0.80      1427
weighted avg       0.83      0.83      0.83      1427



### MultiClassifierDL

We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes. It is designed for multi-label classification purposes. Here we will use MultiClassifierDL as a binary classifier. 

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier = nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
!cat $log_folder/MultiClassifierDLApproach_*

Training started - epochs: 20 - learning_rate: 0.009 - batch_size: 16 - training_examples: 5531 - classes: 2
Epoch 0/20 - 10.94s - loss: 0.5176699 - acc: 0.7431242 - batches: 346
Epoch 1/20 - 2.86s - loss: 0.45592391 - acc: 0.77926546 - batches: 346
Epoch 2/20 - 3.03s - loss: 0.42114297 - acc: 0.8023633 - batches: 346
Epoch 3/20 - 2.72s - loss: 0.39981228 - acc: 0.81558794 - batches: 346
Epoch 4/20 - 2.74s - loss: 0.38461632 - acc: 0.82174736 - batches: 346
Epoch 5/20 - 2.73s - loss: 0.3692872 - acc: 0.83139 - batches: 346
Epoch 6/20 - 2.96s - loss: 0.3546942 - acc: 0.83813405 - batches: 346
Epoch 7/20 - 2.82s - loss: 0.34020808 - acc: 0.8442029 - batches: 346
Epoch 8/20 - 2.65s - loss: 0.3262495 - acc: 0.8541667 - batches: 346
Epoch 9/20 - 2.73s - loss: 0.31264758 - acc: 0.8611413 - batches: 346
Epoch 10/20 - 2.78s - loss: 0.29955116 - acc: 0.8677536 - batches: 346
Epoch 11/20 - 3.14s - loss: 0.2856323 - acc: 0.8763587 - batches: 346
Epoch 12/20 - 2.78s - loss: 0.27050516 - acc: 0.885

In [ ]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings).cache()

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category_array, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    1088
['pos']     339
Name: result, dtype: int64

In [ ]:
preds_df[preds_df.result.apply(len)==2]

,category,text,result,metadata


In [ ]:
preds_df[preds_df.result.apply(len)==0]

,category,text,result,metadata


MultiClassifierDL is a multi-label classifier, so some predictions may include both labels or none of the labels. That can be controlled a bit with `.setThreshold()` parameter during training. For now we will keep not keep zero label predictions and get the highest score as prediction.

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

neg    1088
pos     339
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.83      0.92      0.87       982
         pos       0.76      0.58      0.66       445

    accuracy                           0.81      1427
   macro avg       0.79      0.75      0.76      1427
weighted avg       0.81      0.81      0.80      1427



### Generic Classifier

In [ ]:
!pip install -q tensorflow==2.11.0 tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.4 MB/s eta 0:00:00


In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

GenericClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}


Instructions for updating:
Colocations handled automatically by placer.


generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [ ]:
!cat $log_folder/GenericClassifierApproach_*

Training 25 epochs
Epoch 1/25	0.47s	Loss: 12.082781	ACC: 0.62057424
Epoch 2/25	0.20s	Loss: 9.757496	ACC: 0.7308765
Epoch 3/25	0.17s	Loss: 10.080137	ACC: 0.70859635
Epoch 4/25	0.16s	Loss: 9.551863	ACC: 0.7258654
Epoch 5/25	0.15s	Loss: 9.164324	ACC: 0.7364268
Epoch 6/25	0.16s	Loss: 9.027724	ACC: 0.74325943
Epoch 7/25	0.14s	Loss: 8.727636	ACC: 0.74796796
Epoch 8/25	0.12s	Loss: 8.744375	ACC: 0.7546297
Epoch 9/25	0.15s	Loss: 8.912432	ACC: 0.7493029
Epoch 10/25	0.15s	Loss: 8.547589	ACC: 0.76097566
Epoch 11/25	0.12s	Loss: 8.44485	ACC: 0.7664799
Epoch 12/25	0.13s	Loss: 8.269875	ACC: 0.7712805
Epoch 13/25	0.13s	Loss: 8.1486	ACC: 0.7771728
Epoch 14/25	0.13s	Loss: 8.092053	ACC: 0.7805924
Epoch 15/25	0.12s	Loss: 8.239771	ACC: 0.7713266
Epoch 16/25	0.14s	Loss: 8.102629	ACC: 0.78206545
Epoch 17/25	0.14s	Loss: 7.953369	ACC: 0.78023726
Epoch 18/25	0.14s	Loss: 7.8793	ACC: 0.7892006
Epoch 19/25	0.13s	Loss: 8.0347395	ACC: 0.7873001
Epoch 20/25	0.14s	Loss: 8.058263	ACC: 0.78587306
Epoch 21/25	0.12s	Loss: 

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings).cache()

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.92      0.71      0.81       982
         pos       0.58      0.87      0.70       445

    accuracy                           0.76      1427
   macro avg       0.75      0.79      0.75      1427
weighted avg       0.82      0.76      0.77      1427



### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.19s	Loss: 27.725254	ACC: 0.6654961
Epoch 2/25	0.08s	Loss: 25.898941	ACC: 0.7126407
Epoch 3/25	0.08s	Loss: 25.168678	ACC: 0.7158302
Epoch 4/25	0.12s	Loss: 24.557821	ACC: 0.71773726
Epoch 5/25	0.09s	Loss: 24.27574	ACC: 0.7159288
Epoch 6/25	0.11s	Loss: 23.950256	ACC: 0.7230245
Epoch 7/25	0.11s	Loss: 23.838213	ACC: 0.7277331
Epoch 8/25	0.08s	Loss: 23.467005	ACC: 0.73039645
Epoch 9/25	0.08s	Loss: 23.194754	ACC: 0.7294166
Epoch 10/25	0.08s	Loss: 23.094522	ACC: 0.7323496
Epoch 11/25	0.10s	Loss: 22.869274	ACC: 0.7431279
Epoch 12/25	0.10s	Loss: 22.844965	ACC: 0.73886657
Epoch 13/25	0.08s	Loss: 22.73261	ACC: 0.7425558
Epoch 14/25	0.09s	Loss: 22.71855	ACC: 0.7421086
Epoch 15/25	0.08s	Loss: 22.551315	ACC: 0.7441538
Epoch 16/25	0.10s	Loss: 22.563852	ACC: 0.7396294
Epoch 17/25	0.08s	Loss: 22.706177	ACC: 0.74300295
Epoch 18/25	0.09s	Loss: 22.292599	ACC: 0.7445023
Epoch 19/25	0.11s	Loss: 22.476967	ACC: 0.744818
Epoch 20/25	0.09s	Loss: 22.186666	ACC: 0.7483231
Epoch 21/2

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.74      0.95      0.83       982
         pos       0.72      0.27      0.39       445

    accuracy                           0.74      1427
   macro avg       0.73      0.61      0.61      1427
weighted avg       0.74      0.74      0.70      1427



### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.19s	Loss: 33.426285	ACC: 0.710464
Epoch 2/25	0.08s	Loss: 26.525383	ACC: 0.7141467
Epoch 3/25	0.08s	Loss: 25.768673	ACC: 0.71485686
Epoch 4/25	0.08s	Loss: 25.402172	ACC: 0.7149884
Epoch 5/25	0.08s	Loss: 25.261005	ACC: 0.71813846
Epoch 6/25	0.07s	Loss: 25.00893	ACC: 0.71938133
Epoch 7/25	0.08s	Loss: 25.07841	ACC: 0.7188552
Epoch 8/25	0.07s	Loss: 25.027952	ACC: 0.7224918
Epoch 9/25	0.08s	Loss: 24.82555	ACC: 0.72173554
Epoch 10/25	0.08s	Loss: 24.668526	ACC: 0.726043
Epoch 11/25	0.08s	Loss: 24.557262	ACC: 0.7281671
Epoch 12/25	0.08s	Loss: 24.936146	ACC: 0.72236687
Epoch 13/25	0.08s	Loss: 24.679155	ACC: 0.72639155
Epoch 14/25	0.08s	Loss: 24.659782	ACC: 0.73065287
Epoch 15/25	0.07s	Loss: 24.36838	ACC: 0.7366504
Epoch 16/25	0.08s	Loss: 24.272463	ACC: 0.73647285
Epoch 17/25	0.10s	Loss: 24.354816	ACC: 0.73545355
Epoch 18/25	0.07s	Loss: 24.404793	ACC: 0.73899806
Epoch 19/25	0.08s	Loss: 24.588554	ACC: 0.73354644
Epoch 20/25	0.08s	Loss: 24.478838	ACC: 0.73482877
Epoc

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.72      0.97      0.83       982
         pos       0.75      0.18      0.29       445

    accuracy                           0.73      1427
   macro avg       0.74      0.58      0.56      1427
weighted avg       0.73      0.73      0.66      1427



## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| "Syndrome malin"-like symptoms probably due to interaction between neurolept...|     neg|[{sentence_embeddings, 0, 109,  "Syndrome malin"-like symptoms probably due t...|
| 'Bail-out' bivalirudin use in patients with thrombotic complications unrespo...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin use in patients with t...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| 1 previously untreated patient with plasma-cell leukaemia and 8 patients wit...|     neg|[{sentence_embeddings, 0, 166,  1 previously untreated patient with plasma-ce...|
| A 15-year-old boy had temporary hypertropia, supraduction deficit, ipsilater...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had temporary hypertropia, ...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [ ]:
log_folder="ADE_logs_healthcare_200d"

### ClassifierDLApproach

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(
    stages = [
        classifier_dl
    ])

In [ ]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.89      0.91      0.90       982
         pos       0.79      0.74      0.76       445

    accuracy                           0.86      1427
   macro avg       0.84      0.83      0.83      1427
weighted avg       0.86      0.86      0.86      1427



### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    938
['pos']    488
[]           1
Name: result, dtype: int64

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

neg    938
pos    488
         1
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

                   0.00      0.00      0.00         0
         neg       0.92      0.87      0.89       982
         pos       0.75      0.82      0.78       445

    accuracy                           0.86      1427
   macro avg       0.56      0.57      0.56      1427
weighted avg       0.86      0.86      0.86      1427



### Generic Classifier

In [ ]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [ ]:
#from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(40)\
    .setBatchSize(16)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.92      0.81      0.86       982
         pos       0.67      0.85      0.75       445

    accuracy                           0.82      1427
   macro avg       0.79      0.83      0.80      1427
weighted avg       0.84      0.82      0.82      1427



### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.13s	Loss: 26.04299	ACC: 0.7166259
Epoch 2/25	0.06s	Loss: 24.621386	ACC: 0.7145938
Epoch 3/25	0.06s	Loss: 23.543066	ACC: 0.7224458
Epoch 4/25	0.06s	Loss: 22.724138	ACC: 0.7359796
Epoch 5/25	0.06s	Loss: 22.40437	ACC: 0.7391888
Epoch 6/25	0.06s	Loss: 21.883837	ACC: 0.7514336
Epoch 7/25	0.06s	Loss: 21.752583	ACC: 0.76186347
Epoch 8/25	0.06s	Loss: 21.470306	ACC: 0.75405097
Epoch 9/25	0.07s	Loss: 21.311256	ACC: 0.76252764
Epoch 10/25	0.05s	Loss: 20.965302	ACC: 0.7654146
Epoch 11/25	0.05s	Loss: 20.60025	ACC: 0.7773043
Epoch 12/25	0.05s	Loss: 20.640434	ACC: 0.7726418
Epoch 13/25	0.06s	Loss: 20.576294	ACC: 0.7720697
Epoch 14/25	0.05s	Loss: 20.445593	ACC: 0.77331257
Epoch 15/25	0.06s	Loss: 20.522554	ACC: 0.7706098
Epoch 16/25	0.06s	Loss: 20.306301	ACC: 0.7767322
Epoch 17/25	0.06s	Loss: 20.040346	ACC: 0.7770807
Epoch 18/25	0.06s	Loss: 20.344408	ACC: 0.7768637
Epoch 19/25	0.06s	Loss: 19.989435	ACC: 0.776068
Epoch 20/25	0.06s	Loss: 20.23527	ACC: 0.78023726
Epoch 21/2

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.79      0.93      0.85       982
         pos       0.74      0.46      0.57       445

    accuracy                           0.78      1427
   macro avg       0.76      0.69      0.71      1427
weighted avg       0.77      0.78      0.76      1427



### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.16s	Loss: 31.058128	ACC: 0.6734401
Epoch 2/25	0.08s	Loss: 25.432837	ACC: 0.7160537
Epoch 3/25	0.07s	Loss: 24.786303	ACC: 0.7166785
Epoch 4/25	0.07s	Loss: 24.389965	ACC: 0.72480005
Epoch 5/25	0.07s	Loss: 23.631254	ACC: 0.7323824
Epoch 6/25	0.06s	Loss: 23.308153	ACC: 0.7422796
Epoch 7/25	0.07s	Loss: 22.996788	ACC: 0.75426793
Epoch 8/25	0.07s	Loss: 23.203959	ACC: 0.754097
Epoch 9/25	0.07s	Loss: 22.679329	ACC: 0.7627052
Epoch 10/25	0.06s	Loss: 22.484568	ACC: 0.76962334
Epoch 11/25	0.07s	Loss: 22.288795	ACC: 0.7758904
Epoch 12/25	0.06s	Loss: 22.433739	ACC: 0.7743385
Epoch 13/25	0.07s	Loss: 22.283274	ACC: 0.7797112
Epoch 14/25	0.07s	Loss: 22.252922	ACC: 0.7782381
Epoch 15/25	0.07s	Loss: 22.183752	ACC: 0.775851
Epoch 16/25	0.07s	Loss: 21.952219	ACC: 0.7772188
Epoch 17/25	0.07s	Loss: 22.34063	ACC: 0.7752723
Epoch 18/25	0.07s	Loss: 21.668781	ACC: 0.7882273
Epoch 19/25	0.07s	Loss: 21.848234	ACC: 0.78010577
Epoch 20/25	0.07s	Loss: 21.563772	ACC: 0.7866688
Epoch 21/

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.81      0.92      0.86       982
         pos       0.75      0.52      0.61       445

    accuracy                           0.80      1427
   macro avg       0.78      0.72      0.74      1427
weighted avg       0.79      0.80      0.78      1427



## Bert Sentence Embeddings (sbiobert_base_cased_mli)

Now we will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = nlp.Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| "Syndrome malin"-like symptoms probably due to interaction between neurolept...|     neg|[{sentence_embeddings, 0, 109,  "Syndrome malin"-like symptoms probably due t...|
| 'Bail-out' bivalirudin use in patients with thrombotic complications unrespo...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin use in patients with t...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| 1 previously untreated patient with plasma-cell leukaemia and 8 patients wit...|     neg|[{sentence_embeddings, 0, 166,  1 previously untreated patient with plasma-ce...|
| A 15-year-old boy had temporary hypertropia, supraduction deficit, ipsilater...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had temporary hypertropia, ...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [ ]:
log_folder="ADE_logs_bert"

### ClassifierDLApproach

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(2)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.69      1.00      0.82       982
         pos       0.00      0.00      0.00       445

    accuracy                           0.69      1427
   macro avg       0.34      0.50      0.41      1427
weighted avg       0.47      0.69      0.56      1427



### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    1061
['pos']     366
Name: result, dtype: int64

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

neg    1061
pos     366
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.86      0.93      0.90       982
         pos       0.82      0.67      0.74       445

    accuracy                           0.85      1427
   macro avg       0.84      0.80      0.82      1427
weighted avg       0.85      0.85      0.85      1427



### Generic Classifier

In [ ]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.93      0.89      0.91       982
         pos       0.78      0.86      0.82       445

    accuracy                           0.88      1427
   macro avg       0.86      0.88      0.87      1427
weighted avg       0.89      0.88      0.88      1427



### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.16s	Loss: 21.490278	ACC: 0.7685185
Epoch 2/25	0.06s	Loss: 16.711014	ACC: 0.830841
Epoch 3/25	0.06s	Loss: 15.628862	ACC: 0.8405672
Epoch 4/25	0.06s	Loss: 14.887003	ACC: 0.849616
Epoch 5/25	0.06s	Loss: 14.480483	ACC: 0.8557384
Epoch 6/25	0.06s	Loss: 14.262522	ACC: 0.8616898
Epoch 7/25	0.06s	Loss: 14.280499	ACC: 0.8548506
Epoch 8/25	0.06s	Loss: 13.6470175	ACC: 0.8659512
Epoch 9/25	0.06s	Loss: 13.772535	ACC: 0.8627552
Epoch 10/25	0.07s	Loss: 13.880726	ACC: 0.8626697
Epoch 11/25	0.06s	Loss: 13.81312	ACC: 0.8617819
Epoch 12/25	0.06s	Loss: 13.326372	ACC: 0.87069917
Epoch 13/25	0.06s	Loss: 13.796119	ACC: 0.8640047
Epoch 14/25	0.06s	Loss: 13.78298	ACC: 0.86240005
Epoch 15/25	0.06s	Loss: 13.310244	ACC: 0.8713634
Epoch 16/25	0.06s	Loss: 13.423906	ACC: 0.8658131
Epoch 17/25	0.06s	Loss: 13.366165	ACC: 0.86919326
Epoch 18/25	0.06s	Loss: 13.969917	ACC: 0.8584938
Epoch 19/25	0.08s	Loss: 13.2213955	ACC: 0.86750317
Epoch 20/25	0.07s	Loss: 13.673065	ACC: 0.8612032
Epoch 21

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.87      0.94      0.91       982
         pos       0.84      0.70      0.76       445

    accuracy                           0.87      1427
   macro avg       0.86      0.82      0.84      1427
weighted avg       0.86      0.87      0.86      1427



### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.17s	Loss: 25.204493	ACC: 0.77113587
Epoch 2/25	0.08s	Loss: 18.767488	ACC: 0.82551426
Epoch 3/25	0.07s	Loss: 16.820026	ACC: 0.84232956
Epoch 4/25	0.07s	Loss: 16.319622	ACC: 0.85001713
Epoch 5/25	0.07s	Loss: 15.535469	ACC: 0.8584938
Epoch 6/25	0.07s	Loss: 15.701779	ACC: 0.8542259
Epoch 7/25	0.07s	Loss: 16.520525	ACC: 0.8508194
Epoch 8/25	0.07s	Loss: 15.947368	ACC: 0.85588306
Epoch 9/25	0.07s	Loss: 15.042729	ACC: 0.868437
Epoch 10/25	0.07s	Loss: 14.668651	ACC: 0.86337334
Epoch 11/25	0.07s	Loss: 15.623028	ACC: 0.8593816
Epoch 12/25	0.08s	Loss: 14.756052	ACC: 0.86643785
Epoch 13/25	0.09s	Loss: 15.502031	ACC: 0.85942763
Epoch 14/25	0.09s	Loss: 15.719972	ACC: 0.85640913
Epoch 15/25	0.09s	Loss: 15.671838	ACC: 0.8588489
Epoch 16/25	0.10s	Loss: 15.423007	ACC: 0.85942763
Epoch 17/25	0.08s	Loss: 15.976094	ACC: 0.8509049
Epoch 18/25	0.08s	Loss: 15.1980095	ACC: 0.8609796
Epoch 19/25	0.08s	Loss: 15.021896	ACC: 0.8638205
Epoch 20/25	0.08s	Loss: 15.150192	ACC: 0.86284727

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.87      0.95      0.91       982
         pos       0.86      0.69      0.76       445

    accuracy                           0.87      1427
   macro avg       0.86      0.82      0.83      1427
weighted avg       0.87      0.87      0.86      1427



## DocumentLogRegClassifier

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = nlp.Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = nlp.StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = nlp.Stemmer() \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("stem")

logreg = medical.DocumentLogRegClassifierApproach()\
    .setInputCols("stem")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler, 
    tokenizer,
    normalizer,
    stopwords_cleaner, 
    stemmer, 
    logreg])
doclogreg_model = clf_Pipeline.fit(trainingData)

In [ ]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.85      0.87      0.86       982
         pos       0.70      0.67      0.68       445

    accuracy                           0.81      1427
   macro avg       0.78      0.77      0.77      1427
weighted avg       0.81      0.81      0.81      1427



# Mtsamples Dataset

## Load Dataset

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mtsamples_classifier.csv

In [ ]:
spark_df = spark.read.csv("mtsamples_classifier.csv", header = True)

spark_df.show(10,truncate=100)

+----------------+----------------------------------------------------------------------------------------------------+
|        category|                                                                                                text|
+----------------+----------------------------------------------------------------------------------------------------+
|Gastroenterology| PROCEDURES PERFORMED: Colonoscopy. INDICATIONS: Renewed symptoms likely consistent with active f...|
|Gastroenterology| OPERATION 1. Ivor-Lewis esophagogastrectomy. 2. Feeding jejunostomy. 3. Placement of two right-s...|
|Gastroenterology| PREOPERATIVE DIAGNOSES: 1. Gastroesophageal reflux disease. 2. Chronic dyspepsia. POSTOPERATIVE ...|
|Gastroenterology| PROCEDURE: Colonoscopy. PREOPERATIVE DIAGNOSES: Rectal bleeding and perirectal abscess. POSTOPER...|
|Gastroenterology| PREOPERATIVE DIAGNOSIS: Right colon tumor. POSTOPERATIVE DIAGNOSES: 1. Right colon cancer. 2. As...|
|Gastroenterology| PREOPERATIVE DIAGNOSI

In [ ]:
spark_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- text: string (nullable = true)



In [ ]:
spark_df.count()

638

In [ ]:
spark_df.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|  115|
|       Neurology|  143|
|      Orthopedic|  223|
|Gastroenterology|  157|
+----------------+-----+



In [ ]:
(trainingData, testData) = spark_df.randomSplit([0.8, 0.2], seed = 42)

In [ ]:
trainingData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   95|
|       Neurology|  121|
|      Orthopedic|  188|
|Gastroenterology|  132|
+----------------+-----+



In [ ]:
testData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   20|
|       Neurology|   22|
|      Orthopedic|   35|
|Gastroenterology|   25|
+----------------+-----+



## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



We will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
|                                                                            text|        category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHARGE DIAGNOSIS: Sympto...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Symptomatic cholelithia...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal reflux disease reflux. ...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hiatal hernia, gastroes...|
+--------------------------------------------------------------------------------+-------------

In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
|                                                                            text|        category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF PRESENT ILLNESS: Ms....|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gastrointestinal bleed....|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomiting. HISTORY OF PRESEN...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphagia and hematemesis w...|
+--------------------------------------------------------------------------------+-------------

In [ ]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [ ]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=80)

+--------------------------------------------------------------------------------+
|                                                  sentence_embeddings.embeddings|
+--------------------------------------------------------------------------------+
|[[-0.010433162, 0.0127568655, 0.110687375, 0.1609855, -0.14818177, -0.0508564...|
|[[-0.0055608996, 0.018396137, 0.12305377, 0.13861515, -0.18651922, -0.0755996...|
|[[-0.019461514, 0.060639214, 0.11142021, 0.12826534, -0.16216077, -0.10416892...|
+--------------------------------------------------------------------------------+
only showing top 3 rows



In [ ]:
log_folder="Mt_logs_healthcare_100d"
!mkdir -p $log_folder

### ClassifierDLApproach

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(50)\
        .setLr(0.005)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.78      0.72      0.75        25
       Neurology       0.76      0.73      0.74        22
      Orthopedic       0.89      0.94      0.92        35
         Urology       0.57      0.60      0.59        20

        accuracy                           0.77       102
       macro avg       0.75      0.75      0.75       102
    weighted avg       0.77      0.77      0.77       102



### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(40)\
  .setLr(5e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                        35
['Urology']                           21
['Gastroenterology']                  19
['Neurology']                         17
[]                                     5
['Neurology', 'Orthopedic']            4
['Orthopedic', 'Gastroenterology']     1
Name: result, dtype: int64

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

Orthopedic          36
Urology             21
Gastroenterology    20
Neurology           20
                     5
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

                       0.00      0.00      0.00         0
Gastroenterology       0.95      0.76      0.84        25
       Neurology       0.80      0.73      0.76        22
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.90      0.95      0.93        20

        accuracy                           0.84       102
       macro avg       0.71      0.67      0.69       102
    weighted avg       0.89      0.84      0.86       102



### Generic Classifier

In [ ]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(False)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.92      0.90        25
       Neurology       0.85      0.77      0.81        22
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.94      0.85      0.89        20

        accuracy                           0.88       102
       macro avg       0.89      0.87      0.88       102
    weighted avg       0.88      0.88      0.88       102



### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.09s	Loss: 3.4591072	ACC: 0.253125
Epoch 2/25	0.01s	Loss: 3.2175627	ACC: 0.24166667
Epoch 3/25	0.01s	Loss: 3.0400388	ACC: 0.259375
Epoch 4/25	0.01s	Loss: 2.935348	ACC: 0.27291667
Epoch 5/25	0.01s	Loss: 2.8367057	ACC: 0.3484375
Epoch 6/25	0.01s	Loss: 2.772933	ACC: 0.3875
Epoch 7/25	0.01s	Loss: 2.7623243	ACC: 0.3515625
Epoch 8/25	0.01s	Loss: 2.7234015	ACC: 0.35260418
Epoch 9/25	0.01s	Loss: 2.6715605	ACC: 0.38333336
Epoch 10/25	0.01s	Loss: 2.695961	ACC: 0.35625
Epoch 11/25	0.01s	Loss: 2.6589093	ACC: 0.36822918
Epoch 12/25	0.01s	Loss: 2.6475878	ACC: 0.38177085
Epoch 13/25	0.01s	Loss: 2.6574771	ACC: 0.359375
Epoch 14/25	0.01s	Loss: 2.6605709	ACC: 0.32760417
Epoch 15/25	0.01s	Loss: 2.6249707	ACC: 0.35885417
Epoch 16/25	0.01s	Loss: 2.6098797	ACC: 0.3697917
Epoch 17/25	0.01s	Loss: 2.6222844	ACC: 0.41614586
Epoch 18/25	0.01s	Loss: 2.6214879	ACC: 0.41302085
Epoch 19/25	0.01s	Loss: 2.585606	ACC: 0.45885414
Epoch 20/25	0.01s	Loss: 2.5880198	ACC: 0.4703125
Epoch 21/25

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.83      0.20      0.32        25
       Neurology       1.00      0.05      0.09        22
      Orthopedic       0.37      1.00      0.54        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.40       102
       macro avg       0.55      0.31      0.24       102
    weighted avg       0.55      0.40      0.28       102



### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.10s	Loss: 6.008815	ACC: 0.24270833
Epoch 2/25	0.01s	Loss: 5.29038	ACC: 0.375
Epoch 3/25	0.01s	Loss: 5.417824	ACC: 0.41458336
Epoch 4/25	0.01s	Loss: 5.2801194	ACC: 0.46666664
Epoch 5/25	0.01s	Loss: 5.086564	ACC: 0.51770836
Epoch 6/25	0.01s	Loss: 5.036023	ACC: 0.49427086
Epoch 7/25	0.01s	Loss: 4.9865675	ACC: 0.5234375
Epoch 8/25	0.01s	Loss: 4.865882	ACC: 0.55572915
Epoch 9/25	0.01s	Loss: 4.833692	ACC: 0.5416666
Epoch 10/25	0.01s	Loss: 4.748389	ACC: 0.56041664
Epoch 11/25	0.01s	Loss: 4.7354565	ACC: 0.5609375
Epoch 12/25	0.01s	Loss: 4.663606	ACC: 0.60729164
Epoch 13/25	0.01s	Loss: 4.669315	ACC: 0.5572916
Epoch 14/25	0.01s	Loss: 4.466198	ACC: 0.5942708
Epoch 15/25	0.01s	Loss: 4.528787	ACC: 0.6020833
Epoch 16/25	0.01s	Loss: 4.39523	ACC: 0.6234375
Epoch 17/25	0.01s	Loss: 4.502599	ACC: 0.5875
Epoch 18/25	0.01s	Loss: 4.354523	ACC: 0.66614586
Epoch 19/25	0.01s	Loss: 4.4413395	ACC: 0.6078125
Epoch 20/25	0.01s	Loss: 4.3317204	ACC: 0.64166665
Epoch 21/25	0.01s	Loss: 

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.61      0.88      0.72        25
       Neurology       0.92      0.50      0.65        22
      Orthopedic       0.63      0.97      0.76        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.66       102
       macro avg       0.54      0.59      0.53       102
    weighted avg       0.56      0.66      0.58       102



## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
|                                                                            text|        category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHARGE DIAGNOSIS: Sympto...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Symptomatic cholelithia...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal reflux disease reflux. ...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hiatal hernia, gastroes...|
+--------------------------------------------------------------------------------+-------------

In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
|                                                                            text|        category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF PRESENT ILLNESS: Ms....|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gastrointestinal bleed....|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomiting. HISTORY OF PRESEN...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphagia and hematemesis w...|
+--------------------------------------------------------------------------------+-------------

In [ ]:
log_folder="Mt_logs_healthcare_200d"

### ClassifierDLApproach 

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(50)\
        .setLr(0.005)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.73      0.76      0.75        25
       Neurology       0.94      0.68      0.79        22
      Orthopedic       0.87      0.97      0.92        35
         Urology       0.62      0.65      0.63        20

        accuracy                           0.79       102
       macro avg       0.79      0.77      0.77       102
    weighted avg       0.80      0.79      0.79       102



### MultiClassifierDL



In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.2)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                        32
['Gastroenterology']                  19
['Urology']                           18
['Neurology']                         13
['Neurology', 'Orthopedic']           11
[]                                     4
['Orthopedic', 'Gastroenterology']     3
['Urology', 'Orthopedic']              1
['Urology', 'Gastroenterology']        1
Name: result, dtype: int64

In [ ]:
# We will get the highest score label as result. you can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

Orthopedic          38
Gastroenterology    22
Urology             20
Neurology           18
                     4
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

                       0.00      0.00      0.00         0
Gastroenterology       1.00      0.88      0.94        25
       Neurology       0.89      0.73      0.80        22
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.90      0.90      0.90        20

        accuracy                           0.87       102
       macro avg       0.73      0.69      0.71       102
    weighted avg       0.91      0.87      0.89       102



### Generic Classifier 

In [ ]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.88      0.88        25
       Neurology       0.88      0.68      0.77        22
      Orthopedic       0.85      0.94      0.89        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.86       102
       macro avg       0.87      0.85      0.85       102
    weighted avg       0.86      0.86      0.86       102



### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.33s	Loss: 3.5066636	ACC: 0.21458332
Epoch 2/25	0.01s	Loss: 3.1059568	ACC: 0.21197918
Epoch 3/25	0.01s	Loss: 2.8738773	ACC: 0.2682292
Epoch 4/25	0.01s	Loss: 2.7688437	ACC: 0.3723958
Epoch 5/25	0.01s	Loss: 2.711024	ACC: 0.37708333
Epoch 6/25	0.01s	Loss: 2.7015207	ACC: 0.35572916
Epoch 7/25	0.01s	Loss: 2.65962	ACC: 0.36302084
Epoch 8/25	0.01s	Loss: 2.6975627	ACC: 0.3255208
Epoch 9/25	0.01s	Loss: 2.671928	ACC: 0.36666667
Epoch 10/25	0.01s	Loss: 2.6496096	ACC: 0.3890625
Epoch 11/25	0.01s	Loss: 2.628315	ACC: 0.4296875
Epoch 12/25	0.01s	Loss: 2.577706	ACC: 0.5130209
Epoch 13/25	0.01s	Loss: 2.5484698	ACC: 0.5161458
Epoch 14/25	0.01s	Loss: 2.556626	ACC: 0.44375
Epoch 15/25	0.01s	Loss: 2.5364676	ACC: 0.40104166
Epoch 16/25	0.01s	Loss: 2.5288365	ACC: 0.44479164
Epoch 17/25	0.01s	Loss: 2.5167692	ACC: 0.46666664
Epoch 18/25	0.01s	Loss: 2.4864218	ACC: 0.51770836
Epoch 19/25	0.01s	Loss: 2.4740875	ACC: 0.53802085
Epoch 20/25	0.01s	Loss: 2.4456973	ACC: 0.53125
Epoch 21/2

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.73      0.64      0.68        25
       Neurology       0.80      0.36      0.50        22
      Orthopedic       0.49      0.97      0.65        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.57       102
       macro avg       0.50      0.49      0.46       102
    weighted avg       0.52      0.57      0.50       102



### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.33s	Loss: 5.5190153	ACC: 0.31458333
Epoch 2/25	0.01s	Loss: 5.4710207	ACC: 0.32760417
Epoch 3/25	0.01s	Loss: 5.2188997	ACC: 0.403125
Epoch 4/25	0.01s	Loss: 5.172654	ACC: 0.41666666
Epoch 5/25	0.01s	Loss: 5.084012	ACC: 0.45885414
Epoch 6/25	0.01s	Loss: 4.930412	ACC: 0.49375
Epoch 7/25	0.01s	Loss: 4.817296	ACC: 0.5697917
Epoch 8/25	0.01s	Loss: 4.875178	ACC: 0.5385417
Epoch 9/25	0.01s	Loss: 4.7677846	ACC: 0.5630208
Epoch 10/25	0.01s	Loss: 4.6884985	ACC: 0.58802086
Epoch 11/25	0.01s	Loss: 4.560391	ACC: 0.6302084
Epoch 12/25	0.01s	Loss: 4.577107	ACC: 0.64427084
Epoch 13/25	0.01s	Loss: 4.516133	ACC: 0.6328125
Epoch 14/25	0.01s	Loss: 4.4473352	ACC: 0.64427084
Epoch 15/25	0.01s	Loss: 4.4356766	ACC: 0.6333333
Epoch 16/25	0.01s	Loss: 4.310003	ACC: 0.7026042
Epoch 17/25	0.01s	Loss: 4.2137804	ACC: 0.671875
Epoch 18/25	0.01s	Loss: 4.231407	ACC: 0.6625
Epoch 19/25	0.01s	Loss: 4.1475825	ACC: 0.6979166
Epoch 20/25	0.01s	Loss: 4.117505	ACC: 0.6921875
Epoch 21/25	0.01s	Los

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.61      0.68      0.64        25
       Neurology       0.62      0.82      0.71        22
      Orthopedic       0.71      0.86      0.78        35
         Urology       1.00      0.15      0.26        20

        accuracy                           0.67       102
       macro avg       0.74      0.63      0.60       102
    weighted avg       0.72      0.67      0.63       102



## Bert Sentence Embeddings (sbiobert_base_cased_mli)

We will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = nlp.Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| "Syndrome malin"-like symptoms probably due to interaction between neurolept...|     neg|[{sentence_embeddings, 0, 109,  "Syndrome malin"-like symptoms probably due t...|
| 'Bail-out' bivalirudin use in patients with thrombotic complications unrespo...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin use in patients with t...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| 1 previously untreated patient with plasma-cell leukaemia and 8 patients wit...|     neg|[{sentence_embeddings, 0, 166,  1 previously untreated patient with plasma-ce...|
| A 15-year-old boy had temporary hypertropia, supraduction deficit, ipsilater...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had temporary hypertropia, ...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [ ]:
log_folder="Mt_logs_bert"

### ClassifierDLApproach 

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.69      1.00      0.82       982
         pos       0.00      0.00      0.00       445

    accuracy                           0.69      1427
   macro avg       0.34      0.50      0.41      1427
weighted avg       0.47      0.69      0.56      1427



### MultiClassifierDL 

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']           1105
['pos']            320
['pos', 'neg']       2
Name: result, dtype: int64

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

neg    1105
pos     322
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.85      0.96      0.90       982
         pos       0.87      0.63      0.73       445

    accuracy                           0.85      1427
   macro avg       0.86      0.79      0.81      1427
weighted avg       0.85      0.85      0.85      1427



### Generic Classifier 

In [ ]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(8)\
    .setLearningRate(0.002)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}


Instructions for updating:
Colocations handled automatically by placer.


generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.97      0.78      0.86       982
         pos       0.66      0.94      0.77       445

    accuracy                           0.83      1427
   macro avg       0.81      0.86      0.82      1427
weighted avg       0.87      0.83      0.83      1427



### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.15s	Loss: 21.13333	ACC: 0.766572
Epoch 2/25	0.07s	Loss: 16.368893	ACC: 0.83639127
Epoch 3/25	0.07s	Loss: 15.349411	ACC: 0.84788644
Epoch 4/25	0.06s	Loss: 14.867418	ACC: 0.85245687
Epoch 5/25	0.06s	Loss: 14.650602	ACC: 0.84846514
Epoch 6/25	0.07s	Loss: 14.117679	ACC: 0.8622225
Epoch 7/25	0.07s	Loss: 14.229368	ACC: 0.8596512
Epoch 8/25	0.07s	Loss: 14.207264	ACC: 0.85831624
Epoch 9/25	0.06s	Loss: 13.872311	ACC: 0.8642611
Epoch 10/25	0.07s	Loss: 13.727418	ACC: 0.8672335
Epoch 11/25	0.06s	Loss: 13.906328	ACC: 0.8612032
Epoch 12/25	0.06s	Loss: 13.330467	ACC: 0.86634576
Epoch 13/25	0.06s	Loss: 13.524981	ACC: 0.86497134
Epoch 14/25	0.06s	Loss: 13.245172	ACC: 0.8724287
Epoch 15/25	0.06s	Loss: 13.209822	ACC: 0.8719421
Epoch 16/25	0.06s	Loss: 13.807668	ACC: 0.85880286
Epoch 17/25	0.06s	Loss: 13.469735	ACC: 0.8705216
Epoch 18/25	0.06s	Loss: 13.28601	ACC: 0.8689236
Epoch 19/25	0.06s	Loss: 13.518498	ACC: 0.8654185
Epoch 20/25	0.06s	Loss: 13.609458	ACC: 0.86555004
Epoc

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.91      0.88      0.89       982
         pos       0.75      0.82      0.78       445

    accuracy                           0.86      1427
   macro avg       0.83      0.85      0.84      1427
weighted avg       0.86      0.86      0.86      1427



### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.17s	Loss: 26.084478	ACC: 0.7603575
Epoch 2/25	0.08s	Loss: 18.10065	ACC: 0.82657963
Epoch 3/25	0.08s	Loss: 16.449371	ACC: 0.8495239
Epoch 4/25	0.08s	Loss: 16.356663	ACC: 0.8433094
Epoch 5/25	0.07s	Loss: 15.976372	ACC: 0.85663277
Epoch 6/25	0.08s	Loss: 15.681889	ACC: 0.8557844
Epoch 7/25	0.08s	Loss: 16.313326	ACC: 0.8522333
Epoch 8/25	0.08s	Loss: 15.932546	ACC: 0.8525884
Epoch 9/25	0.08s	Loss: 15.615725	ACC: 0.8584938
Epoch 10/25	0.09s	Loss: 15.672672	ACC: 0.8541864
Epoch 11/25	0.09s	Loss: 15.023664	ACC: 0.8615583
Epoch 12/25	0.08s	Loss: 15.67601	ACC: 0.854364
Epoch 13/25	0.08s	Loss: 16.418985	ACC: 0.8506353
Epoch 14/25	0.09s	Loss: 14.837163	ACC: 0.86958784
Epoch 15/25	0.08s	Loss: 14.787988	ACC: 0.86395204
Epoch 16/25	0.08s	Loss: 15.6768	ACC: 0.8584017
Epoch 17/25	0.08s	Loss: 15.403945	ACC: 0.8599997
Epoch 18/25	0.08s	Loss: 15.095381	ACC: 0.8607626
Epoch 19/25	0.08s	Loss: 15.419379	ACC: 0.85969067
Epoch 20/25	0.08s	Loss: 15.751317	ACC: 0.8569418
Epoch 21/2

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.88      0.94      0.91       982
         pos       0.85      0.71      0.77       445

    accuracy                           0.87      1427
   macro avg       0.86      0.83      0.84      1427
weighted avg       0.87      0.87      0.87      1427



## DocumentLogRegClassifier

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

logreg = medical.DocumentLogRegClassifierApproach()\
    .setInputCols("token")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = nlp.Pipeline(stages=[document_assembler,
                                tokenizer,
                                logreg])

doclogreg_model = clf_Pipeline.fit(trainingData)

In [ ]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.85      0.91      0.88       982
         pos       0.76      0.66      0.71       445

    accuracy                           0.83      1427
   macro avg       0.81      0.78      0.79      1427
weighted avg       0.83      0.83      0.83      1427

